In [24]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score, recall_score, precision_score, balanced_accuracy_score
import scipy

%matplotlib inline

In [2]:
df = pd.read_csv('2020.1 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.csv', skiprows=9)
df = df[df['Salario mensual BRUTO (en tu moneda local)'] < 1_000_000]
df = df[df['Años en la empresa actual'] < 40]
df = df[(df['Salario mensual BRUTO (en tu moneda local)'] >= 10_000) & (df['Salario mensual BRUTO (en tu moneda local)'] <= 1_000_000)]
df.head()

,Me identifico,Tengo,Dónde estás trabajando,Años de experiencia,Años en la empresa actual,Años en el puesto actual,¿Gente a cargo?,Nivel de estudios alcanzado,Estado,Carrera,...,¿Tenés algún tipo de discapacidad?,¿Sentís que esto te dificultó el conseguir trabajo?,¿A qué eventos de tecnología asististe en el último año?,Cantidad de empleados,Actividad principal,¿La recomendás como un buen lugar para trabajar?,¿Cómo calificás las políticas de diversidad e inclusión?,¿A cuántos kilómetros de tu casa queda la oficina?,Beneficios extra,"¿Cuáles considerás que son las mejores empresas de IT para trabajar en este momento, en tu ciudad?"
0,Hombre,40,Catamarca,15.0,0.0,0.0,0,Universitario,Completado,Licenciatura en Sistemas de Información,...,NaN,NaN,NaN,201-500,Otras industrias,6,6,4.0,"Comidas pagas / subvencionadas, Estacionamient...",NaN
1,Mujer,35,Catamarca,15.0,10.0,3.0,5,Posgrado,Completado,Licenciatura en Sistemas de Información,...,NaN,No,NaN,11-50,Servicios / Consultoría de Software / Digital,5,8,2.0,Abono de celular y/o Internet,NaN
2,Hombre,39,Chaco,10.0,5.0,5.0,6,Universitario,Incompleto,Ingeniería en Sistemas de Información,...,NaN,NaN,Ekoparty,11-50,Servicios / Consultoría de Software / Digital,10,10,1.0,"Capacitaciones y/o cursos, Clases de idiomas, ...",NaN
3,Hombre,42,Chaco,18.0,3.0,3.0,0,Universitario,Completado,Licenciatura en Análisis de Sistemas,...,NaN,No,NaN,11-50,Producto basado en Software,5,3,-1.0,"Horarios flexibles, Sueldo dolarizado, WFH (Tr...",NaN
4,Hombre,29,Chaco,4.0,0.0,0.0,1,Universitario,Completado,Ingeniería Electrónica,...,NaN,NaN,"Pynea, Autodefensa digital",201-500,Otras industrias,8,9,5.0,"Abono de celular y/o Internet, Comidas pagas /...",NaN


In [12]:
df.columns

Index(['Me identifico', 'Tengo', 'Dónde estás trabajando',
       'Años de experiencia', 'Años en la empresa actual',
       'Años en el puesto actual', '¿Gente a cargo?',
       'Nivel de estudios alcanzado', 'Estado', 'Carrera', 'Universidad',
       'Realizaste cursos de especialización',
       '¿Contribuís a proyectos open source?', '¿Programás como hobbie?',
       'Trabajo de', 'Plataformas', 'Lenguajes de programación',
       'Frameworks, herramientas y librerías', 'Bases de datos',
       'QA / Testing', 'IDEs', '¿Qué SO usás en tu laptop/PC para trabajar?',
       '¿Y en tu celular?', '¿Tenés guardias?', 'Cuánto cobrás por guardia',
       '¿Porcentaje, bruto o neto?', 'Tipo de contrato',
       'Salario mensual BRUTO (en tu moneda local)',
       'Salario mensual NETO (en tu moneda local)',
       '¿Qué tan conforme estás con tu sueldo?',
       'Cómo creés que está tu sueldo con respecto al último semestre',
       'Recibís algún tipo de bono', 'A qué está atado el bono',


In [7]:
df['Nivel de estudios alcanzado'].unique()

array(['Universitario', 'Posgrado', 'Terciario', 'Secundario',
       'Doctorado', 'Posdoctorado', 'Primario'], dtype=object)

In [14]:
df['Tipo de contrato'].value_counts()

Full-Time                                                 4902
Part-Time                                                  325
Tercerizado (trabajo a través de consultora o agencia)     206
Remoto (empresa de otro país)                              184
Freelance                                                  149
Name: Tipo de contrato, dtype: int64

In [18]:
df['Cantidad de empleados'].unique()

array(['201-500', '11-50', '1-10', '5001-10000', '101-200', '10001+',
       '2001-5000', '51-100', '1001-2000', '501-1000'], dtype=object)

In [17]:
df = df[df['Tipo de contrato'] == 'Full-Time']

In [22]:
def prepare_row(row):
    return pd.Series({
        'log(salary)': np.log(row['Salario mensual BRUTO (en tu moneda local)'] / 40),
        'male': row['Me identifico'] == 'Hombre',
        'age': row['Tengo'],
        'age_squared': row['Tengo']**2,
        'education': {
            'Primario 1': 0,
            'Secundario 1': 1,
            'Terciario 1': 2,
            'Universitario 1': 2,
            'Posgrado 1': 3,
            'Doctorado 1': 3,
            'Posdoctorado 1': 3,
            'Primario 0': 0,
            'Secundario 0': 0,
            'Terciario 0': 1,
            'Universitario 0': 1,
            'Posgrado 0': 2,
            'Doctorado 0': 3,
            'Posdoctorado 0': 3,
        }[f"{row['Nivel de estudios alcanzado']} {int(row['Estado'] == 'Completado')}"],
        'current_enterprise_years': row['Años en la empresa actual'],
        'current_enterprise_years_squared': row['Años en la empresa actual'] ** 2,
        'enterprise_size': {
            '201-500': 2,
            '11-50': 0,
            '1-10': 0,
            '5001-10000': 4,
            '101-200': 1,
            '10001+': 4,
            '2001-5000': 4,
            '51-100': 1,
            '1001-2000': 4,
            '501-1000': 3,
        }[row['Cantidad de empleados']],
    })

df = df.apply(prepare_row, axis=1)

In [23]:
df_male = df[df['male'] == True].drop('male', axis=1)
df_female = df[df['male'] != True].drop('male', axis=1)

In [26]:
LinearRegression().fit(df_male.drop('log(salary)', axis=1), df_male['log(salary)']).score(df_male.drop('log(salary)', axis=1), df_male['log(salary)'])

0.19728766518202767

In [27]:
LinearRegression().fit(df_female.drop('log(salary)', axis=1), df_female['log(salary)']).score(df_female.drop('log(salary)', axis=1), df_female['log(salary)'])

0.15809294237238314

In [31]:
m_male = LinearRegression().fit(df_male.drop('log(salary)', axis=1), df_male['log(salary)'])
m_female = LinearRegression().fit(df_female.drop('log(salary)', axis=1), df_female['log(salary)'])
u = np.mean(m_male.predict(df_female.drop('log(salary)', axis=1)) - m_female.predict(df_female.drop('log(salary)', axis=1)))
E = np.mean(m_male.predict(df_male.drop('log(salary)', axis=1)))  - np.mean(m_male.predict(df_female.drop('log(salary)', axis=1)))
gpg = (np.mean(df_male['log(salary)']) - np.mean(df_female['log(salary)'])) / np.mean(df_male['log(salary)'])
gpg * u / (u + E)

0.025439128393805518